In [ ]:
import numpy as np
from tqdm import tqdm
import cv2
import pickle
from natsort import natsorted
import matplotlib.pyplot as plt
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers

## Data Loading

## Model Creation and Training

In [ ]:
def simple_ANN_model_creation(layers=[], learning_rate = 0.001):

    '''
    The input will be a np.array of length 50, with each index representing a tag
    The value of each index is the amount of saves that tag has received

    The output will be a np.array of length 50, with each index representing a tag
    The value of each index is either 1 or 0 indicating whether the tag should be recommended or not
    '''

    model = tf.keras.Sequential()
    input_layer = tf.keras.Input(shape=(50,))
    for layer in layers:
        model.add(tf.keras.layers.Dense(layer, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(50, activation='sigmoid'))

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='sigmoid_cross_entropy_with_logits', metrics=['accuracy'])

    model.summary()

    return model

In [ ]:
def train_model(X, Y, model, validation_split = 0.2, batch_size = 16, epochs = 100):

  print("Training Model...")

  with tf.device('/device:GPU:0'):
    history = model.fit(
        x = X,
        y = Y,
        validation_split = validation_split,
        batch_size = batch_size,
        epochs = epochs
    )

  print("best train loss: " + str(min(history.history['loss'])) + "\n" + "best train acc: " + str(max(history.history['accuracy'])))
  print("best test loss: " + str(min(history.history['val_loss'])) + "\n" + "best test acc: " + str(max(history.history['val_accuracy'])))

  return history


## Script

In [ ]:
model = simple_ANN_model_creation([128, 128, 128])

In [ ]:
history = train_model(X, Y, model)

## Explainability

In [ ]:
epochs = 100
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()